In [1]:
import bs4, requests, re
from urllib.request import Request, urlopen
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup
import numpy as np
from selenium.webdriver.support.wait import WebDriverWait
import time
import random

In [2]:
country_web = requests.get('https://www.planespotters.net/airlines/country', headers={'User-Agent': 'Google Chrome'}).text

In [3]:
country_index = bs4.BeautifulSoup(country_web,'lxml')

In [4]:
#Dùng regular expression để tìm các href có /country/
country_links = country_index.find_all('a', href = re.compile("/airlines/")) 

In [5]:
country_links

[<a href="/airlines/country">Airlines by Country</a>,
 <a href="/airlines/afghanistan">Afghanistan</a>,
 <a href="/airlines/afghanistan">8</a>,
 <a href="/airlines/albania">Albania</a>,
 <a href="/airlines/albania">8</a>,
 <a href="/airlines/algeria">Algeria</a>,
 <a href="/airlines/algeria">8</a>,
 <a href="/airlines/american-samoa">American Samoa</a>,
 <a href="/airlines/american-samoa">1</a>,
 <a href="/airlines/angola">Angola</a>,
 <a href="/airlines/angola">15</a>,
 <a href="/airlines/antigua-and-barbuda">Antigua And Barbuda</a>,
 <a href="/airlines/antigua-and-barbuda">8</a>,
 <a href="/airlines/argentina">Argentina</a>,
 <a href="/airlines/argentina">39</a>,
 <a href="/airlines/armenia">Armenia</a>,
 <a href="/airlines/armenia">20</a>,
 <a href="/airlines/aruba">Aruba</a>,
 <a href="/airlines/aruba">7</a>,
 <a href="/airlines/australia">Australia</a>,
 <a href="/airlines/australia">79</a>,
 <a href="/airlines/austria">Austria</a>,
 <a href="/airlines/austria">32</a>,
 <a href="/

In [6]:
country_list = []
for link in country_links:
    country_list.append(link['href'])

In [7]:
country_set = set(country_list)

In [8]:
country_set

{'/airlines/afghanistan',
 '/airlines/albania',
 '/airlines/algeria',
 '/airlines/american-samoa',
 '/airlines/angola',
 '/airlines/antigua-and-barbuda',
 '/airlines/argentina',
 '/airlines/armenia',
 '/airlines/aruba',
 '/airlines/australia',
 '/airlines/austria',
 '/airlines/azerbaijan',
 '/airlines/bahamas',
 '/airlines/bahrain',
 '/airlines/bangladesh',
 '/airlines/barbados',
 '/airlines/belarus',
 '/airlines/belgium',
 '/airlines/belize',
 '/airlines/benin',
 '/airlines/bermuda',
 '/airlines/bhutan',
 '/airlines/bolivia',
 '/airlines/bosnia-and-herzegovina',
 '/airlines/botswana',
 '/airlines/brazil',
 '/airlines/brunei',
 '/airlines/bulgaria',
 '/airlines/burkina-faso',
 '/airlines/cambodia',
 '/airlines/cameroon',
 '/airlines/canada',
 '/airlines/cape-verde',
 '/airlines/cayman-islands',
 '/airlines/central-african-republic',
 '/airlines/chad',
 '/airlines/chile',
 '/airlines/china',
 '/airlines/colombia',
 '/airlines/comoros',
 '/airlines/congo',
 '/airlines/congo-the-democrati

In [9]:
country_info_pages = []
for country in country_set:
    country_info_pages.append('https://www.planespotters.net/'+country)

In [10]:
country_info_pages

['https://www.planespotters.net//airlines/chile',
 'https://www.planespotters.net//airlines/mexico',
 'https://www.planespotters.net//airlines/cyprus',
 'https://www.planespotters.net//airlines/algeria',
 'https://www.planespotters.net//airlines/new-caledonia',
 'https://www.planespotters.net//airlines/israel',
 'https://www.planespotters.net//airlines/macedonia-the-former-yugoslav-republic-of',
 'https://www.planespotters.net//airlines/turks-and-caicos-islands',
 'https://www.planespotters.net//airlines/sao-tome-and-principe',
 'https://www.planespotters.net//airlines/egypt',
 'https://www.planespotters.net//airlines/guyana',
 'https://www.planespotters.net//airlines/brunei',
 'https://www.planespotters.net//airlines/iraq',
 'https://www.planespotters.net//airlines/tonga',
 'https://www.planespotters.net//airlines/aruba',
 'https://www.planespotters.net//airlines/uruguay',
 'https://www.planespotters.net//airlines/bangladesh',
 'https://www.planespotters.net//airlines/philippines',
 '

In [11]:
airline = set()
for country in country_info_pages:
    country_detail = requests.get(country, headers={'User-Agent': 'Google Chrome'}).text
    soup = bs4.BeautifulSoup(country_detail,'lxml')
    airlines_links = soup.find_all('a',href = re.compile("^/airline/"))
    for i in airlines_links:
        airline.add(i['href'])

In [12]:
len(airline)

4054

In [13]:
airline_info_pages = []
for i in airline:
    airline_info_pages.append('https://www.planespotters.net/'+i)

In [14]:
airline_info_pages

['https://www.planespotters.net//airline/Aerosur',
 'https://www.planespotters.net//airline/Aerotours-Dominicana',
 'https://www.planespotters.net//airline/Air-Brasil-1991',
 'https://www.planespotters.net//airline/Toki-Air',
 'https://www.planespotters.net//airline/ExecuJet-Scandinavia',
 'https://www.planespotters.net//airline/Africa-Charter-Airline',
 'https://www.planespotters.net//airline/Cayman-Airways',
 'https://www.planespotters.net//airline/Avcon-Jet-Malta',
 'https://www.planespotters.net//airline/Moalem-Aviation',
 'https://www.planespotters.net//airline/Red-Air-Dominican-Republic',
 'https://www.planespotters.net//airline/Tunisair',
 'https://www.planespotters.net//airline/HiSky',
 'https://www.planespotters.net//airline/Aeromar',
 'https://www.planespotters.net//airline/Afriqiyah-Airways',
 'https://www.planespotters.net//airline/Aligiulia',
 'https://www.planespotters.net//airline/World-Focus-Airways',
 'https://www.planespotters.net//airline/ACG-Air-Cargo-Global',
 'htt

In [15]:
result = pd.DataFrame(columns = ['AC', 'In Service', 'Parked', 'Total Current','Future','Historic','Avg Age','Total'])

In [16]:
proxies = {'http': 'http://144.168.240.98:9999', 'https': 'http://95.216.249.203:83'}

In [17]:
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [18]:
proxies = get_proxies()

In [19]:
from itertools import cycle
import traceback

In [20]:
proxy_pool = cycle(proxies)

In [ ]:
for page in airline_info_pages:
    proxy = next(proxy_pool)
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome("chromedriver.exe",options=chrome_options)
    chrome_options.add_argument(f'--proxy-server={proxy}')
    driver.implicitly_wait(random.randint(2,20))
    driver.get(page)
    time.sleep(random.randint(2,20))
    df = pd.read_html(driver.page_source, attrs = {'class':'fleet_matrix'})[0]
    df.columns = ['AC', 'In Service', 'Parked', 'Total Current','Future','Historic','Avg Age','Total']
    df = df[(df['Total Current'].notna())|(df['Future'].notna())]
    time.sleep(random.randint(2,20))
    WebDriverWait(page, random.randint(2,20))
    soup = BeautifulSoup(driver.page_source,'lxml')
    ac_list = []
    for i in soup.find_all('th',attrs = {'class': re.compile("subtype")}):
        ac_list.append(i.text)
    df = df[df['AC'].isin(ac_list)]
    result.append(df)